In [1]:
!pip install statsmodels

     |████████████████████████████████| 10.8MB 9.1MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"MONTHLY_DMD_v1"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [5]:
data = df[['PRODUCT_ID','ORDER_MONTH','ORDER_QTY']]

In [6]:
data['ORDER_MONTH'] = pd.to_datetime(data['ORDER_MONTH'])

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2948 entries, 0 to 2947
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   PRODUCT_ID   2948 non-null   object        
 1   ORDER_MONTH  2948 non-null   datetime64[ns]
 2   ORDER_QTY    2948 non-null   int32         
dtypes: datetime64[ns](1), int32(1), object(1)
memory usage: 57.7+ KB


In [8]:
data = data.set_index('ORDER_MONTH')

In [9]:
data.sort_index(inplace=True)

In [10]:
data

,PRODUCT_ID,ORDER_QTY
ORDER_MONTH,,
2019-01-01,Product_001,171000
2019-01-01,Product_087,38850
2019-01-01,Product_019,85050
2019-01-01,Product_086,39100
2019-01-01,Product_085,30100
...,...,...
2024-07-01,Product_001,182050
2024-07-01,Product_078,49950
2024-07-01,Product_074,34900


In [11]:
# Group data by 'product_id'
grouped_data = data.groupby('PRODUCT_ID')

In [12]:
# Initialize an empty DataFrame for forecast results
test_results = pd.DataFrame(columns=['product_id','order_date', 'test_forecast_orders','rse', 'rmse','r2_score'])

In [13]:
# Initialize an empty DataFrame for forecast results
forecast_results = pd.DataFrame(columns=['product_id','order_date', 'forecast_order_qty'])

In [14]:
for product_id, group in grouped_data:
    # Sort data by order date
    group.sort_index(inplace=True)
    
    # Split data into train and test sets (80% train, 20% test)
    train_size = int(0.8 * len(group))
    train_data, test_data = group.iloc[:train_size], group.iloc[train_size:]
    
    # Create SARIMAX model
    model = SARIMAX(train_data['ORDER_QTY'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit(disp=False)
    
    # Forecast the next period
    test_model = model_fit.get_forecast(steps=len(test_data))
    test_df = test_model.predicted_mean
    
    # Calculate RMSE for order quantity
    #rmse_order_qty = np.sqrt(mean_squared_error(test_data['ORDER_QTY'], forecast_order_qty))
    
    # Evaluate the model
    rse = np.sqrt(((test_df - test_data) ** 2).sum().sum() / (test_data.shape[0] * test_data.shape[1] - len(model_fit.params)))
    rmse = np.sqrt(mean_squared_error(test_data['ORDER_QTY'], test_df))
    r2 = r2_score(test_data['ORDER_QTY'], test_df)

    
    date_list = test_df.index.to_list()
    
    new_row = {
        'product_id': product_id,
        'order_date': date_list,
        'test_forecast_orders': test_df,
        'rse': rse,
        'rmse': rmse,
        'r2_score': r2
    }
    
    df_new = pd.DataFrame(new_row)
    test_results=pd.concat([test_results,df_new],ignore_index=True)
    
    # Forecast future values
    forecast = model_fit.get_forecast(steps=25)#, steps=n_forecast)
    forecast_df = forecast.predicted_mean
    
    date_list2= forecast_df.index.to_list()
    
    new_row2 = {
        'product_id': product_id,
        'order_date': date_list2,
        'forecast_order_qty': forecast_df,
    }
    
    df_new2 = pd.DataFrame(new_row2)
    
    forecast_results=pd.concat([forecast_results,df_new2],ignore_index=True)

In [15]:
test_results

,product_id,order_date,test_forecast_orders,rse,rmse,r2_score
0,Product_001,2023-06-01,161665.764497,0.0,13419.128597,0.437121
1,Product_001,2023-07-01,159095.529480,0.0,13419.128597,0.437121
2,Product_001,2023-08-01,137134.458693,0.0,13419.128597,0.437121
3,Product_001,2023-09-01,129698.559422,0.0,13419.128597,0.437121
4,Product_001,2023-10-01,150450.869655,0.0,13419.128597,0.437121
...,...,...,...,...,...,...
611,Product_100,2024-03-01,38844.787461,0.0,6671.280251,-0.933250
612,Product_100,2024-04-01,26714.851510,0.0,6671.280251,-0.933250
613,Product_100,2024-05-01,31208.430045,0.0,6671.280251,-0.933250
614,Product_100,2024-06-01,38355.519216,0.0,6671.280251,-0.933250


In [ ]:
forecast_results

In [20]:
test_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 616 entries, 0 to 615
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   product_id            616 non-null    object        
 1   order_date            616 non-null    datetime64[ns]
 2   test_forecast_orders  616 non-null    float64       
 3   rse                   616 non-null    float64       
 4   rmse                  616 non-null    float64       
 5   r2_score              616 non-null    float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 29.0+ KB


In [22]:
sf_df = my_session.createDataFrame(test_results)
sf_df.write.mode("overwrite").save_as_table("ORDER_FORECAST_TEST")
my_session.table("ORDER_FORECAST_TEST").show()

------------------------------------------------------------------------------------------------------------------
|"product_id"  |"order_date"         |"test_forecast_orders"  |"rse"  |"rmse"              |"r2_score"           |
------------------------------------------------------------------------------------------------------------------
|Product_001   |2023-06-01 00:00:00  |161665.76449723775      |0.0    |13419.128596617618  |0.43712070636724043  |
|Product_001   |2023-07-01 00:00:00  |159095.52947978332      |0.0    |13419.128596617618  |0.43712070636724043  |
|Product_001   |2023-08-01 00:00:00  |137134.45869321877      |0.0    |13419.128596617618  |0.43712070636724043  |
|Product_001   |2023-09-01 00:00:00  |129698.55942177543      |0.0    |13419.128596617618  |0.43712070636724043  |
|Product_001   |2023-10-01 00:00:00  |150450.86965511978      |0.0    |13419.128596617618  |0.43712070636724043  |
|Product_001   |2023-11-01 00:00:00  |146835.26561759668      |0.0    |13419.128

In [23]:
sf_df = my_session.createDataFrame(forecast_results)
sf_df.write.mode("overwrite").save_as_table("ORDER_FORECAST")
my_session.table("ORDER_FORECAST").show()

-------------------------------------------------------------
|"product_id"  |"order_date"         |"forecast_order_qty"  |
-------------------------------------------------------------
|Product_001   |2023-06-01 00:00:00  |161665.76449723775    |
|Product_001   |2023-07-01 00:00:00  |159095.52947978332    |
|Product_001   |2023-08-01 00:00:00  |137134.45869321877    |
|Product_001   |2023-09-01 00:00:00  |129698.55942177543    |
|Product_001   |2023-10-01 00:00:00  |150450.86965511978    |
|Product_001   |2023-11-01 00:00:00  |146835.26561759668    |
|Product_001   |2023-12-01 00:00:00  |146980.05314085158    |
|Product_001   |2024-01-01 00:00:00  |163679.86302398637    |
|Product_001   |2024-02-01 00:00:00  |156849.57526935427    |
|Product_001   |2024-03-01 00:00:00  |163978.67954817638    |
-------------------------------------------------------------

